In [1]:
import numpy as np
import pandas as pd

In [2]:
# Carga de datos
df=pd.read_csv('../data/retail_sales_dataset.csv')

In [3]:
df.head()

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


### Transformación de Datos
Crea nuevas columnas: Basándonos en los datos existentes, crea nuevas columnas que sean útiles para el análisis. Por ejemplo, calcula el ingreso total por venta y normaliza las ventas.


Clasifica los datos: Crea una columna que clasifique las ventas en categorías significativas (e.g., ‘Alta’, ‘Media’, ‘Baja’).

In [16]:
# Calcular el ingreso total de cada venta
df['Total Income'] = df['Quantity'] * df['Price per Unit']

# Normalizar las ventas para facilitar la comparación entre transacciones
df['Normalized Sales'] = (df['Total Income'] - df['Total Income'].mean()) / df['Total Income'].std()
df[['Total Income', 'Normalized Sales']].head()


,Total Income,Normalized Sales
0,150,-0.546431
1,1000,0.971433
2,30,-0.760718
3,500,0.078572
4,100,-0.635717


In [15]:
# Definición de condiciones y categorías
conditions = [
    df['Total Income'] > df['Total Income'].quantile(0.75),
    df['Total Income'].between(df['Total Income'].quantile(0.25), df['Total Income'].quantile(0.75)),
    df['Total Income'] < df['Total Income'].quantile(0.25)
]
categories = ['Alta', 'Media', 'Baja']

# Crear la columna 'Sales Category' usando np.select para asignar cada categoría a las condiciones correspondientes
df['Sales Category'] = np.select(conditions, categories, default='Media')  # Default es 'Media' si ninguna condición se cumple
df[['Sales Category']].head()



,Sales Category
0,Media
1,Alta
2,Baja
3,Media
4,Media


### Agrupación y Agregación
Agrupación por múltiples columnas: Realiza agrupaciones por categorías como producto y tienda, producto y mes, etc.

Aplicar funciones de agregación: Utiliza funciones como sum, mean, count, min, max, std, y var para obtener estadísticas descriptivas de cada grupo.

In [18]:
grouped_df = df.groupby(['Product Category', 'Date']).agg({
    'Total Income': ['sum', 'mean', 'count', 'min', 'max', 'std', 'var'],
    'Quantity': 'sum'
}).reset_index()
grouped_df.columns = ['Product Category', 'Date', 'Total Income Sum', 'Total Income Mean', 'Transaction Count', 
                    'Total Income Min', 'Total Income Max', 'Total Income Std', 'Total Income Var', 'Total Quantity']
df.head()

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,Total Income,Normalized Sales,Sales Category,Income Deviation
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,150,-0.546431,Media,-317.475570
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,1000,0.971433,Alta,556.752137
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,30,-0.760718,Baja,-428.786550
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,500,0.078572,Media,56.752137
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,100,-0.635717,Media,-367.475570


### Análisis Personalizado con apply
Función personalizada: Aplica funciones personalizadas para realizar análisis específicos que no se pueden lograr con las funciones de agregación estándar.

Ejemplo de uso avanzado: Calcula la desviación de cada venta respecto a la media de su grupo.

In [ ]:
# Definir una función que calcule la desviación respecto a la media
def deviation_from_mean(x):
    return x - x.mean()

# Calcular la desviación en cada grupo de producto
df['Income Deviation'] = df.groupby('Product Category')['Total Income'].transform(deviation_from_mean)
df[['Income Deviation']].head()


,Income Deviation
0,-317.475570
1,556.752137
2,-428.786550
3,56.752137
4,-367.475570


### Transformación de Datos:

Se calculó el ingreso total por venta (Total Income) multiplicando la cantidad por el precio unitario. Esto permite analizar los ingresos generados en cada transacción.
Para facilitar la comparación de ingresos entre transacciones de diferentes escalas, se normalizó el ingreso total, creando una columna de ventas normalizadas (Normalized Sales).
Se creó una columna de clasificación de ventas (Sales Category) para categorizar cada venta en niveles significativos (Alta, Media, Baja), usando los percentiles del ingreso total.
Agrupación y Agregación:

Se agruparon los datos por categoría de producto y fecha, lo que permite observar patrones de ventas y comportamiento por producto a lo largo del tiempo.
Sobre cada grupo se aplicaron funciones de agregación (como suma, media, conteo, mínimo, máximo, desviación estándar y varianza) para obtener estadísticas descriptivas de las ventas e ingresos.
Análisis Personalizado:

Para un análisis más detallado, se calculó la desviación de cada ingreso total respecto a la media del grupo de su categoría de producto. Esto se logró aplicando una función personalizada, lo cual nos da una medida de variabilidad dentro de cada grupo.
Comentarios y Legibilidad:

El código está documentado con comentarios claros en cada paso, y se organizaron las operaciones de manera que el flujo de análisis sea fácil de seguir y mantener.